In [ ]:
from dotenv import load_dotenv
import os
from pymongo import MongoClient

load_dotenv()
db_password = os.environ.get("DB_PASSWORD")
cluster_uri = os.environ.get("MONGODB_URI")
client = MongoClient(cluster_uri)
db = client["MSCI446_DB"]
collection = db["Load_Forecast"]